In [1]:
from pyspark.ml.feature import VectorAssembler

In [9]:
ctx = SparkSession.builder \
    .master("spark://h:7077") \
    .appName("bosch spark") \
    .getOrCreate()
file_dir = "/home/h/kaggle_fun/bosch/raw_data/"

- 1. change dataframe to vector
- 2. missing values count
- 3. fill missing values
- 4. plot feature point distribution
- 5. normalize the feature
- 6. adjust positive negative ratio
- 7. train model
- 8. evaluate model matric

In [3]:
def load_csv_df(file_name):
    return ctx.read.csv(file_dir + file_name, header=True, inferSchema=True)

def check_p_n_cnt(input_df, col_name, pos):
    tmp_table_name = "check_p_n_input_df"
    input_df.registerTempTable(tmp_table_name)
    total = ctx.sql("select count(*) as tcnt from %s" % tmp_table_name)
    pos = ctx.sql("select count(*) as pcnt from %s where %s = %s" %(tmp_table_name, col_name, pos))
    print "positive/total : %s/%s" % (pos.collect()[0]["pcnt"], total.collect()[0]["tcnt"])

def adjust_p_n_ratio(input_df, cate_col_name, scale, raw_ratio):
    tmp_table_name = "adjust_p_n_input_df"
    input_df.registerTempTable(tmp_table_name)
    adjusted_df = ctx.sql("""select * from %s where %s = 1 or (%s = 0 and rand() < %s*%s)"""
                          % (tmp_table_name, cate_col_name, cate_col_name, scale, raw_ratio))
    return adjusted_df

def df_to_vec(input_df, ignore_list=[]):
    assembler = VectorAssembler(
        inputCols=[x for x in input_df.columns if x not in ignore_list],
        outputCol='features')
    return assembler.transform(input_df)

def get_corr_matrix(vec, method="pearson"):
    return vec.stat.corr(vec)

In [4]:
train_numer_df = load_csv_df("train_numeric.csv")
test_numer_df = load_csv_df("test_numeric.csv")

In [7]:
#check_p_n_cnt(train_numer_df, "Response", "1") #6879/1183747
adjusted_train_num = adjust_p_n_ratio(train_numer_df, "Response", 1.2, float(6879)/1183747)
print "pos", adjusted_train_num.filter(adjusted_train_num.Response == 1).count()
print "neg", adjusted_train_num.filter(adjusted_train_num.Response == 0).count()

pos 6879
neg 8304


In [18]:
#train_num_vec = df_to_vec(train_numer_df, ["Id"])
#train_num_vec.corr(method="pearson")
ignore_list = []
assembler = VectorAssembler(
        inputCols=[x for x in train_numer_df.columns if x not in ignore_list],
        outputCol='features')
#print type(train_num_vec)
train_numer_vec = assembler.transform(train_numer_df)

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
labelIndexer = StringIndexer(inputCol="Response", outputCol="label").fit(train_numer_df)
#featureIndexer =\
#    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)
DecisionTreeClassifier dt = new DecisionTreeClassifier().setLabelCol("labelIndexed").setFeaturesCol("features");
Pipeline pipeline = new Pipeline().setStages(new PipelineStage[] {assembler, indexer, dt});
# Train model
PipelineModel model = pipeline.fit(trainingData);

# Make predictions
DataFrame predictions = model.transform(testData)
lrModel = lr.fit(train_num_vec)

In [ ]:
corr_mat = get_corr_matrix(train_numer_df)

In [ ]:
from pyspark.mllib.stat import Statistics
import numpy as np
from pyspark.mllib.linalg import Vectors
train_num_vec = train_numer_df.rdd.map(lambda data: Vectors.dense([float(c) for c in data]))
summary = Statistics.colStats(train_num_vec)

#Statistics.corr(train_numer_df)

In [ ]:
train_numer_df.head